# FireRoad053 Model

In [5]:
## if you have not installed package "CSV" or "JSON", please uncomment the line below and execute it
# using Pkg; Pkg.add("CSV")

using DataFrames, CSV
students = CSV.read("finaldata/students_data.csv", DataFrame)
classes = CSV.read("finaldata/parsedsp21_actual_classes.csv", DataFrame)

┌ Warning: thread = 1 warning: only found 11 / 12 columns around data row: 7. Filling remaining columns with `missing`
└ @ CSV /Users/laurenah/.julia/packages/CSV/YUbbG/src/file.jl:603


,Column1,gir_attribute,hass_attribute,id,prereqs
,String,String?,String?,String,String?
1,8.01,PHY1,missing,8.01,missing
2,8.02,PHY2,missing,8.02,GIR:CAL1 and GIR:PHY1
3,18.01,CAL1,missing,18.01,missing
4,18.02,CAL2,missing,18.02,GIR:CAL1
5,7.014,BIOL,missing,7.014,missing
6,7.016,BIOL,missing,7.016,missing
7,3.091,CHEM,missing,3.091,missing
8,5.111,CHEM,missing,5.111,missing
9,5.07,REST,missing,5.07,5.12


# Model

In [29]:
#--- Model specification
using JuMP, DataFrames, Gurobi
M = 1000000
S = nrow(classes)
T = 8
min_units = 12 #change this for each student!
max_units = 60 #change this for each student!
min_workload = 40 #change this for each student!
max_workload = 70 #change this for each student!

model = Model(with_optimizer(Gurobi.Optimizer))

@variable(model, x[1:S, 1:T], Bin)
@objective(model, Max, sum())
#@objective(model, Max, sum(utility[i, :] * sum(x[i,j] for j = 1:T) for i = 1:S))

Academic license - for non-commercial use only - expires 2021-07-07


50×8 Array{VariableRef,2}:
 x[1,1]   x[1,2]   x[1,3]   x[1,4]   x[1,5]   x[1,6]   x[1,7]   x[1,8]
 x[2,1]   x[2,2]   x[2,3]   x[2,4]   x[2,5]   x[2,6]   x[2,7]   x[2,8]
 x[3,1]   x[3,2]   x[3,3]   x[3,4]   x[3,5]   x[3,6]   x[3,7]   x[3,8]
 x[4,1]   x[4,2]   x[4,3]   x[4,4]   x[4,5]   x[4,6]   x[4,7]   x[4,8]
 x[5,1]   x[5,2]   x[5,3]   x[5,4]   x[5,5]   x[5,6]   x[5,7]   x[5,8]
 x[6,1]   x[6,2]   x[6,3]   x[6,4]   x[6,5]   x[6,6]   x[6,7]   x[6,8]
 x[7,1]   x[7,2]   x[7,3]   x[7,4]   x[7,5]   x[7,6]   x[7,7]   x[7,8]
 x[8,1]   x[8,2]   x[8,3]   x[8,4]   x[8,5]   x[8,6]   x[8,7]   x[8,8]
 x[9,1]   x[9,2]   x[9,3]   x[9,4]   x[9,5]   x[9,6]   x[9,7]   x[9,8]
 x[10,1]  x[10,2]  x[10,3]  x[10,4]  x[10,5]  x[10,6]  x[10,7]  x[10,8]
 x[11,1]  x[11,2]  x[11,3]  x[11,4]  x[11,5]  x[11,6]  x[11,7]  x[11,8]
 x[12,1]  x[12,2]  x[12,3]  x[12,4]  x[12,5]  x[12,6]  x[12,7]  x[12,8]
 x[13,1]  x[13,2]  x[13,3]  x[13,4]  x[13,5]  x[13,6]  x[13,7]  x[13,8]
 ⋮                                            

# Constraints

In [30]:
# 1. Each student takes (or gets credit for) all of the Science/Math GIRs.
# Math GIR
@constraint(model, math, sum(x[3, t] + x[4, t] for t = 1:T) == 1)

# Physics GIR
@constraint(model, physics, sum(x[1, t] + x[2, t] for t = 1:T) == 1)

# Chemistry GIR
@constraint(model, chem, sum(x[7, t] + x[8, t] for t = 1:T) == 1)

# Biology GIR
@constraint(model, bio, sum(x[5, t] + x[6, t] for t = 1:T) == 1)

# 2. Each student takes 8 HASS subjects.
@constraint(model, hass, sum(x[s, t] for s = 42:50 for t = 1:T) >= 8)

# 3. Each student satisfies the requirements of her major (and minor).


# 4. No subject is taken prior to its prerequisites.
@constraint(model, prereq802[t in 2:T], sum(x[1, j] + x[3, j] for j in 1:t-1) >= 2 * x[2,t])

# 5. Each student takes 216 units beyond GIRs.
@constraint(model, units, sum(x[s, t] * classes[s, 8] for s = 1:S for t = 1:T) >= 208)

# 6. No two subjects taken in Spring 2021 should overlap in time.

    
# 7. Students cannot take the same subject twice.
#@constraint(model, once, sum(x[s, t] for s = 1:S for t = 1:T) <= 1)

# 8. Each student can specify if they want to take a specific subject in the future.
#@constraint(model, req[s in 1:S], sum(x[s, t] for t in 1:T) >= 1 - M * (1 - REQ CSV[s]))

# 9. Each student can specify which semester they want to take a subject.
#@constraint(model, reqsem[s in 1:S, t in 1:T], x[s, t] >= 1 - M * (1 - REQSEM CSV[s, t]))

# 10. Each student can specify which semester they do not want to take a subject. 
#@constraint(model, reqnot[s in 1:S, t in 1:T], x[s, t] <= 1 + M * (1 - REQNOT CSV[s, t]))
    
# 11. The minimum and maximum number of units that each student wants to take each semester.
#@constraint(model, minunits[t in 1:T], sum(x[s, t] * classes[s, 8] for s in 1:S) >= min_units)
#@constraint(model, maxunits[t in 1:T], sum(x[s, t] * classes[s, 8] for s in 1:S) <= max_units)
    
# 12. The minimum and maximum workload (in number of hours) that each student wants to take on each semester.
#@constraint(model, minwork[t in 1:T], sum(x[s, t] * classes[s, 10] for s in 1:S) >= min_workload)
#@constraint(model, maxwork[t in 1:T], sum(x[s, t] * classes[s, 10] for s in 1:S) <= max_workload)

8-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 prereq802[1] : -2 x[2,1] ≥ 0.0
 prereq802[2] : x[1,1] - 2 x[2,2] + x[3,1] ≥ 0.0
 prereq802[3] : x[1,1] + x[1,2] - 2 x[2,3] + x[3,1] + x[3,2] ≥ 0.0
 prereq802[4] : x[1,1] + x[1,2] + x[1,3] - 2 x[2,4] + x[3,1] + x[3,2] + x[3,3] ≥ 0.0
 prereq802[5] : x[1,1] + x[1,2] + x[1,3] + x[1,4] - 2 x[2,5] + x[3,1] + x[3,2] + x[3,3] + x[3,4] ≥ 0.0
 prereq802[6] : x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] - 2 x[2,6] + x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] ≥ 0.0
 prereq802[7] : x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] + x[1,6] - 2 x[2,7] + x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] + x[3,6] ≥ 0.0
 prereq802[8] : x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] + x[1,6] + x[1,7] - 2 x[2,8] + x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] + x[3,6] + x[3,7] ≥ 0.0

In [4]:
#--- Write codes here to print your solutions
optimize!(model)
@show termination_status(model)
@show objective_value(model)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 400 columns and 0 nonzeros
Model fingerprint: 0x8d7293a6
Variable types: 0 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

User-callback calls 21, time in user-callback 0.00 sec
termination_status(model) = MathOptInterface.OPTIMAL
objective_value(model) = 0.0


0.0

In [5]:
display("text/csv", value.(x)) # could also display JSON

-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
